# Lecture 9 - Documenter.jl and Continuous Deployment

- In Lecture 7, you saw how to create documentation for your Julia packages using Documenter.jl.


- In this lecture, we'll show you how to host that documentation online as a website, and how to keep it automatically up to date using continuous deployment.


- Before showing you how to host your documentation online, however, we'll first demonstrate how to build it and host it from your own computer. This will be useful when developing your documentation, and it will make clear some of the concepts required for later hosting your documentation on the web.

## Contents

1. [A Documenter.jl Workflow for Developing and Hosting Documentation Locally](#local)
2. [An Aside on Continuous Delivery and Continuous Deployment (CD)](#cd)
3. [A GitHub Actions CD Pipeline for Hosting Your Documentation on GitHub Pages](#pages)
4. [Further Reading](#further-reading)

## 1. A Documenter.jl Workflow for Developing and Hosting Documentation Locally <a class="anchor" id="local"></a>

In this example, we'll create some documentation for the package Lorenz.jl, which we created in last week's demonstration.

### Step-by-Step Guide

1. Make sure you have [Revise.jl](https://timholy.github.io/Revise.jl/stable/) installed in your default Julia environment. This is a useful Julia package which picks up your source code changes so they can be used immediately in your current Julia session, without the need to restart your session and reload the code. I include `using Revise` in my `~/.julia/config/startup.jl` file so that Revise.jl is loaded automatically every time I start a Julia session.


2. As before, create a directory `docs` in the package you want to document, as follows:

```
Lorenz/
├── docs/
│   ├── src/
│   └── make.jl
├── src/
│   ├── Lorenz.jl
│   │
│   ...
...
```

The directory `docs/src` will contain the markdown files for your documentation, while the file `docs/make.jl` will contain the Documenter.jl commands required to build your documentation. Create an empty file `index.md` inside `/src`.


3. Create a new Julia environment especially for your documentation. Change to the `docs` directory and in the Pkg REPL do:

```
pkg> activate .
pkg> add Documenter
pkg> dev ~/code/Lorenz
```

The `docs` directory should now contain a `Project.toml` file with Documenter and Lorenz as dependencies.

4. Now we're ready to create some documentation. To get started, add the following lines to `make.jl`:

```julia
using Documenter, Lorenz

makedocs(sitename="Lorenz.jl")
```


5. In the Julia REPL, do `include("make.jl")`. This will run the commands in `docs/make.jl` and create the webpages for your documentation. Those files will be placed inside the directory `docs/build` (you should add this file to your `.gitignore` file so that it doesn't get committed to GitHub).


6. To view the documentation you've just created, change to the `docs/build` directory and start up a local webserver. This is most easily achieved with Python:

```
python3 -m http.server --bind localhost
```

But you can also use the Julia LiveServer package:

```
julia -e 'using LiveServer; serve(dir="docs/build")'
```

Now you can view your (locally built and hosted) documentation in your web browser! As you can see, it's currently empty.

7. As you develop your documentation, simply call `include("make.jl")` in the Julia REPL each time you wish to view your changes and then refresh your browser.

## 2. An Aside on Continuous Delivery and Continuous Deployment (CD) <a class="anchor" id="cd"></a>

- Last week you learned about continuous integration (CI), a software engineering practice in which developers regularly (~daily) commit their changes to a central repository and make use of tools such as automated testing pipelines to verify its correctness.


- Two concepts closely related to continuous integration are continuous delivery and continuous deployment. In fact, you will typically hear developers talking about "CI/CD" rather than just CI or CD in isolation.


- Continuous delivery refers to the practice where engineers regularly deliver software, which is then deployed periodically in small releases, typically on a short cycle (~week).


- Continuous deployment is similar to continuous delivery, except that the deployment process is also fully automated and occurs in realtime.  


- Since the software we are writing does not typically provide a service, in the sense of a traditional client-server model, deployment in general is well beyond the scope of this course. However, one thing we can and should deploy (continuously) is documentation!

## 3. A GitHub Actions CD Pipeline for Hosting Your Documentation on GitHub Pages <a class="anchor" id="pages"></a>

- [GitHub Pages](https://pages.github.com/) allows you to host the documentation of your project on the internet for free. 


- Not only that, you can also use [GitHub Actions](https://github.com/features/actions) to set up a continuous deployment pipeline to automatically keep your documentation up to date with changes in your repository.


### Step-by-Step Guide


1. First, add the `deploydocs()` command to your `make.jl` file. It should now look something like this:

```julia
using Documenter, Lorenz

makedocs(
    sitename = "Lorenz.jl",
    pages = [
        "Home" => "index.md",
        "Functions" => "functions.md",
    ],
)

deploydocs(
    repo = "github.com/white-alistair/Lorenz.jl.git",
)

```


2. Then, create a new workflow configuration file in your repo called `.github/workflows/documentation.yml` with the following contents:

```yaml
name: Documentation

on:
  push:
    branches:
      - main

jobs:
  build:
    permissions:
      contents: write
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2
      - uses: julia-actions/setup-julia@v1
      - name: Install dependencies
        run: julia --project=docs/ -e 'using Pkg; Pkg.develop(PackageSpec(path=pwd())); Pkg.instantiate()'
      - name: Build and deploy
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
        run: julia --project=docs/ docs/make.jl
```

Based on this file, every time you push to your repository's `main` branch, your documentation will be automatically built and committed to the branch `gh-pages` (if this branch doesn't already exist, it will be created).


3. Commit these changes and push them to GitHub. Verify that the GitHub Actions job has completed successfully and the docs have been pushed to the `gh-pages` branch.


4. To publish your documentation to the web using GitHub Pages, go to your repository's settings and navigate to the section called "Pages". There, select `gh-pages` as the branch for your GitHub Pages site. After a few minutes, you should be able to view your documentation using the link provided. 


5. Now, everytime you commit a change to your documentation, it will be automatically built *and* deployed!

## 4. Further Reading <a class="anchor" id="further-reading"></a>
1. [Julia Documentation Guidelines](https://docs.julialang.org/en/v1/manual/documentation/)
2. [Documenter.jl Package Guide](https://juliadocs.github.io/Documenter.jl/stable/man/guide/)
3. [CI/CD (Wikipedia)](https://en.wikipedia.org/wiki/CI/CD)